In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
database_name = ''

In [ ]:
session.sql(f"""
    CREATE OR REPLACE TABLE {database_name}.PUBLIC.DDL_EXPORT(
        OBJECT_NAME STRING,
        DDL STRING
    );
""").collect()

In [ ]:
table_df = session.sql(f"""
    SELECT TABLE_CATALOG||'.'||TABLE_SCHEMA||'.'||TABLE_NAME AS OBJECT_NAME
    FROM {database_name}.INFORMATION_SCHEMA.TABLES
    WHERE TABLE_SCHEMA NOT IN ('INFORMATION_SCHEMA','PUBLIC','SNOWFLAKE_TRACKER');
""").to_pandas()

print(table_df['OBJECT_NAME'])

In [ ]:
for table_name in table_df['OBJECT_NAME']:
    try:
        ddl_result = session.sql(f"""
            SELECT GET_DDL('TABLE', '{table_name}') AS DDL
        """).collect()
        
        ddl_text = ddl_result[0]['DDL']
        
        # Insert into output table
        session.sql(f"""
            INSERT INTO DDL_EXPORT (OBJECT_NAME, DDL)
            VALUES ('{table_name}', '{ddl_text}')
        """).collect()
        
        print(f"✓ Exported: {table_name}")
        
    except Exception as e:
        print(f"✗ Error with {table_name}: {e}")

print(f"\nDone! Exported {len(table_df)} tables.")